In [ ]:
#RpostgreSQL install package
install.packages("RPostgreSQL")

library(DBI)
library(RPostgreSQL)
library(dplyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [10]:
#create variable, and assign database connection function to it
con <- dbConnect(
  PostgreSQL(), #let r configurate PostgreSQL automatically/with default settings
  host = "tiny.db.elephantsql.com", #host as elephantsql
  dbname = "cjfscdod",
  port = 5432,
  user = "cjfscdod",
  password = "YUwk9TswZEtyPNvRIVClhMmbF5rWm57k"
)

con

<PostgreSQLConnection>

In [11]:
#mock-up tables (favorite band, genre, bridge)
best_band <- data.frame(
  band_id = 1:5,
  bands = c("Vildhjarta", "Tesseract", "Dance Gavin Dance", "Owane", "Playboi Carti"),
  ratings = c(9, 8, 8, 10, 6)
)

music_genres <- data.frame(
  genre_id = 1:7,
  genre = c("Progressive Metal", "Djent", "Thall", "Jazz-Fusion", "Post-Hardcore", "Progressive Rock", "Hip-hop")
)

music_bridge <- data.frame(
  band_id = c(1, 1, 2, 2, 3, 3, 4, 4, 5),
  genre_id = c(1, 2, 1, 3, 4, 5, 4, 6, 7)
)

In [12]:
#write table to database
dbWriteTable(con, "best_band", best_band, row.names=FALSE)
dbWriteTable(con, "music_genres", genres, row.names=FALSE)
dbWriteTable(con, "music_bridge", bridge, row.names=FALSE)

[1] TRUE

[1] TRUE

[1] TRUE

In [13]:
# list all tables from the database server
dbListTables(con)

[1] "best_band"    "music_genres" "music_bridge"

In [14]:
#get data from the db server

my_band_info <- dbGetQuery(con,
                "SELECT
                    ba.bands,
                    ba.ratings,
                    mg.genre
                FROM best_band AS ba
                JOIN music_bridge AS br ON ba.band_id = br.band_id
                JOIN music_genres AS mg ON br.genre_id = mg.genre_id
                ORDER BY 1")

my_band_info

,bands,ratings,genre
,<chr>,<dbl>,<chr>
1,Dance Gavin Dance,8,Jazz-Fusion
2,Dance Gavin Dance,8,Post-Hardcore
3,Owane,10,Progressive Rock
4,Owane,10,Jazz-Fusion
5,Playboi Carti,6,Hip-hop
6,Tesseract,8,Thall
7,Tesseract,8,Progressive Metal
8,Vildhjarta,9,Djent
9,Vildhjarta,9,Progressive Metal


In [15]:
#get data from db server
#user dplyr to manipulate

my_band <- dbGetQuery(con, "SELECT * FROM best_band")

my_band %>%
  mutate(label = c("Century Media", "Century Media", "Rise Record", "Independent", "Interscope"))

,band_id,bands,ratings,label
,<int>,<chr>,<dbl>,<chr>
1,1,Vildhjarta,9,Century Media
2,2,Tesseract,8,Century Media
3,3,Dance Gavin Dance,8,Rise Record
4,4,Owane,10,Independent
5,5,Playboi Carti,6,Interscope


In [16]:
#close connection
dbDisconnect(con)

[1] TRUE